In [1]:
import pandas as pd
import numpy as np
import string
import json
import random

In [2]:
## Static Config
locations = [
    {
        "column": "Woreda",
        "name": "level1"
    },
    {
        "column": "Kebele",
        "name": "level2"
    }
]

latlong = ["latitude", "longitude"]
charts = [
    "water", 
    "sanitation", 
    "hygiene", 
    "health care waste service level",
    "environmental cleaning service level"
]
default = {
    "water" : ["Advanced", "Basic", "Limited", "No Service"],
    "sanitation": ["Advanced", "Basic", "Limited", "No Service"],
    "hygiene": ["Advanced", "Basic", "Limited", "No Service"],
    "health care waste service level": ["Advanced", "Basic", "Limited", "No Service"],
    "environmental cleaning service level": ["Advanced", "Basic", "Limited", "No Service"]
}

## Marker Detail
marker = {
    "name" : "Name_of_Health_facility",
    "color": [],
    "detail": []
}

main = {
    'name':'Health Facility',
    'column': 'Name_of_Health_facility',
    'indicators': [{
        'name': 'Water',
        'column': 'Water'
    },{
        'name': 'Sanitation',
        'column': 'Sanitation'
    },{
        'name': 'Hygiene',
        'column': 'Hygiene'
    }]
}


In [3]:
df = pd.read_csv('eth_health_data.csv')

In [4]:
## Delete Column with no name
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [5]:
## Fill Column
#default_column = ["Identifier", "Woreda", "Kebele", "Name_of_Health_facility", "Latitude", "Longitude", "Type_of_healthy_facility"]
#for column in df.columns:
#    if column not in default_column and column != "Environmental Cleaning Service Level":
#        df[column] = df[column].apply(lambda x: random.choice(df[column].dropna().unique()))

In [6]:
### Fill Empty Integer Values
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_num = df.select_dtypes(include=numerics).drop(columns=['Latitude', 'Longitude'])
df_num = df_num.fillna(0.0).astype(np.int32)
df[list(df_num)] = df_num

In [7]:
### Replace Empty String Values
df_str = df.select_dtypes(include=['object']).fillna("")
for strcol in list(df_str):
    df_str[strcol] = df_str[strcol].str.strip()
df[list(df_str)] = df_str

In [8]:
## Generate Settings 
chars =list(string.ascii_uppercase)
chars_col = chars + [x+y for x in chars for y in chars]
keyname = lambda x,y: {a:y[b] if (a != 'data_point_id') else 'data_point_id' for b, a in enumerate(x)}

columns_length = len(list(df))
column_names = list(df)

In [9]:
### Generate configs
index = chars_col[:columns_length]
configs = keyname(index, column_names)

marker_config = {}
chart_config = []
latlong_config = {}
for key in configs:
    if(configs[key].lower() in charts):
        chart_config.append({'key': key, 'name': configs[key], 'value': default[configs[key].lower()]})
    if(configs[key].lower() in latlong):
        latlong_config.update({configs[key].lower(): key})
    if(configs[key].lower() == marker['name'].lower()):
        marker_config = { "name" : key}

#### Locations
loc_config = {}
for loc in locations:
    column_index = ""
    for key in configs:
        if loc['column'].lower() == configs[key].lower():
            column_index = key
    loc_config.update({loc['name'].lower(): column_index})

#### Marker
marker_color = [];
for mk in marker['color']:
    column = None
    andColumn = None
    for key in configs:
        if configs[key].lower() == mk['column'].lower():
            column = key
        if mk['and'] != None and configs[key].lower() == mk['and'].lower():
            andColumn = key
    marker_color.append({
        "name": mk['name'],
        "column": column,
        "action": mk['action'],
        "type": mk['type'],
        "value": mk['value'],
        "and": andColumn,
        "and_value": mk['and_value'],
        "color": mk['color']
    })
marker_config['color'] = marker_color

#### Marker Detail
marker_detail = [];
for mk in marker['detail']:
    column = None
    andColumn = None
    for key in configs:
        if configs[key].lower() == mk['column'].lower():
            column = key
        if mk['and'] != None and configs[key].lower() == mk['and'].lower():
            andColumn = key
    marker_detail.append({
        "name": mk['name'],
        "column": column,
        "action": mk['action'],
        "type": mk['type'],
        "value": mk['value'],
        "and": andColumn,
        "and_value": mk['and_value']
    })
marker_config['detail'] = marker_detail
    
## Main Table
main_config = {'name':main['name']}
indicators = []
for key in configs:
    if configs[key] == main['column']:
        main_config.update({'key':key})
    for i in main['indicators']:
        if configs[key] == i['column']:
            indicators.append({'name':i['name'],'key':key})
main_config.update({'indicators':indicators})
            
configs['charts'] = chart_config
configs['marker'] = marker_config 
configs['locations'] = loc_config
configs['latlong'] = latlong_config
configs['main'] = main_config

In [10]:
### Replace Dataset Columns
df.columns = index

In [11]:
configs

{'A': 'Woreda',
 'B': 'Kebele',
 'C': 'Name_of_Health_facility',
 'D': 'Latitude',
 'E': 'Longitude',
 'F': 'Type_of_healthy_facility',
 'G': 'Description of Water Supply in Health Facilities',
 'H': 'Water',
 'I': 'Latrine_Description',
 'J': 'Sanitation',
 'K': 'Physically_Separate_Latrines_for_Male_Female_Patients',
 'L': 'Facilitated_Access_for_Disabled',
 'M': 'Latrine_condition',
 'N': '992194004|Is_there_hand_washing_facility_with_in_5m_of_the_toilet',
 'O': 'Hygiene',
 'P': 'Is_hand_washing_facility_operating_i.e_with_water',
 'Q': 'Is_Soap_available_at_each_hand_washing_stand',
 'R': 'Is_there_a_toilet_for_staff',
 'charts': [{'key': 'H',
   'name': 'Water',
   'value': ['Advanced', 'Basic', 'Limited', 'No Service']},
  {'key': 'J',
   'name': 'Sanitation',
   'value': ['Advanced', 'Basic', 'Limited', 'No Service']},
  {'key': 'O',
   'name': 'Hygiene',
   'value': ['Advanced', 'Basic', 'Limited', 'No Service']}],
 'marker': {'name': 'C', 'color': [], 'detail': []},
 'location

In [12]:
data = list(df.T.to_dict().values())
data[:2]

[{'A': 'Shashemene',
  'B': 'Mararo',
  'C': 'Mararo',
  'D': 7.21461843,
  'E': 38.67594311,
  'F': 'Health Post',
  'G': 'No water supply in premises',
  'H': 'No Service',
  'I': 'No Latrine',
  'J': 'No Service',
  'K': 'No',
  'L': 'No',
  'M': 'Filthy ,unused',
  'N': 'No Facility',
  'O': 'No Service',
  'P': 'Yes',
  'Q': 'No',
  'R': 'No'},
 {'A': 'Shashemene',
  'B': 'Jigesa qorke',
  'C': 'Jigesa',
  'D': 7.205212947,
  'E': 38.66626716,
  'F': 'Health Post',
  'G': 'No water supply in premises',
  'H': 'No Service',
  'I': 'No Latrine',
  'J': 'No Service',
  'K': 'No',
  'L': 'No',
  'M': 'Filthy ,unused',
  'N': 'No Facility',
  'O': 'No Service',
  'P': 'Yes',
  'Q': 'No',
  'R': 'No'}]

In [13]:
### to json
df.to_json('../resources/js/data/eth_health_data.json', orient="records", indent=2)

with open('../resources/js/data/eth_health_config.json', 'w') as outfile:
    json.dump(configs, outfile)